https://github.com/jamescalam/transformers/blob/main/course/training/06_nsp_training.ipynb

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving w2v_pm_cui_5K.csv to w2v_pm_cui_5K.csv


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 4.7 MB 48.8 MB/s 
     |████████████████████████████████| 1.2 MB 65.7 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 596 kB 7.7 MB/s 
     |████████████████████████████████| 6.6 MB 46.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=807ecea93c5d1db8a9ad4c73f23d9b241e031fcafd6f0afd8474180df9ee20e6
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import InputExample
from tqdm.auto import tqdm  # so we see progress bar
import re
import pandas as pd

data= pd.read_csv('/content/drive/MyDrive/dataset/bio_all_word_5K_del.csv')

train_samples = []
for row in tqdm(range(data.shape[0])):
  
  train_samples.append(InputExample(
      texts=[data["sentence_a"][row], data["sentence_b"][row]],
      label=float(data["cosine"][row])
  ))
#總筆數

  0%|          | 0/4894 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import models, SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer('all-mpnet-base-v2')
#model = SentenceTransformer('bert-base-uncased')
#model = SentenceTransformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract')

#model = SentenceTransformer('emilyalsentzer/Bio_ClinicalBERT')#distilbert-base-nli-mean-tokens
#model = SentenceTransformer('dmis-lab/biosyn-sapbert-bc5cdr-disease')

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

for i in range(1,6):
  print(i)
  #Tune the model
  model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=i, warmup_steps=100, output_path='/content/drive/MyDrive/model/word/SBERT_word_'+str(i))# epochs建議2-4 太大會過擬合 

1


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

2


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

import torch
torch.cuda.empty_cache()

In [ ]:
model = SentenceTransformer('./sbert_test')


In [35]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import pickle
from scipy.stats import spearmanr
from scipy import spatial
from scipy.stats import spearmanr
import numpy as np
import scipy.stats
#model = SentenceTransformer('./sbert_test')

#model = SentenceTransformer('/content/drive/MyDrive/model/cui/PubMedBERT_cui_5')
model = SentenceTransformer('/content/drive/MyDrive/model/word/PubMedBERT_word_2')

#model = SentenceTransformer('bert-base-uncased')
#model = SentenceTransformer('all-mpnet-base-v2')
#model = SentenceTransformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract')

#model = SentenceTransformer('dmis-lab/biosyn-sapbert-bc5cdr-disease')
#model = SentenceTransformer('dmis-lab/biobert-base-cased-v1.2')
#model = SentenceTransformer('emilyalsentzer/Bio_ClinicalBERT')


# Two lists of sentences
df= pd.read_csv('/content/drive/MyDrive/ans.csv',encoding='big5')#pair_t2d_fin

sentences1 = df['pair1'].tolist()
sentences2 = df['pair2'].tolist()


#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

print(len(sentences1))
print(len(sentences2))
print(len(embeddings1))
print(len(embeddings2))

lst_cos=[]
lst_eu=[]
for i in range(len(sentences1)):

  v1 = embeddings1[i].cpu().detach().numpy()
  v2 = embeddings2[i].cpu().detach().numpy()

  result = 1 - spatial.distance.cosine(v1, v2)
  lst_cos.append(result)

  result_e = spatial.distance.euclidean(v1, v2)
  lst_eu.append(result_e*(-1))#*(-1) 讓數值從小到大(不相關-相關)



df_new = df.assign(cosine = lst_cos,  euclidean = lst_eu  )
# #Compute cosine-similarits
# cosine_scores = util.cos_sim(embeddings1, embeddings2)


# cos=[]
# lst_eu=[]
# #Output the pairs with their score
# for i in range(len(sentences1)):
#     #print("{} \t\t {} \t\t COS Score: {:.4f}".format(sentences2[i], sentences1[i], cosine_scores[i][i]))
#     cos.append(float(cosine_scores[i][i]))

# print(len(cos))

# print(df_new['cosine'])
# print(df_new['euclidean'])


y = df['mean'].tolist()
#y_m = df['median'].tolist()
# correlation, p = spearmanr(df_new['cosine'], y_m)#cosine
# print('Correlation_cos:', round(correlation, 4))

# y.pop(96)
# df_new['cosine'].pop(96)
# print(len(y),len(df_new['cosine']))

# correlation, p = spearmanr(df_new['cosine'], y)#cosine
# print('Correlation_cos:', round(correlation, 2))
# print('Correlation_cos:', round(correlation, 4))

nor = scipy.stats.shapiro(df_new['cosine'])
print(nor[1])
if nor[1]<0.05:
    print("不是常態")
else:
    print("常態")
    pearson = scipy.stats.pearsonr(df_new['cosine'], y)
    if pearson[1] < 0.05:
        print(round(pearson[0], 2),"*")
    else:
        print(round(pearson[0], 2))


# correlation, p = spearmanr(df_new['euclidean'], y)#euclidean
# print('Correlation_eu:', round(correlation, 4))

# print("========================")

# correlation, p = spearmanr(df_new['cosine'][0:50], y[0:50])#cosine
# print('Correlation_cos:', round(correlation, 2))

# correlation, p = spearmanr(df_new['euclidean'][0:50], y[0:50])#euclidean
# print('Correlation_eu:', round(correlation, 4))

# print("========================")

# correlation, p = spearmanr(df_new['cosine'][50:100], y[50:100])#cosine
# print('Correlation_cos:', round(correlation, 2))

# correlation, p = spearmanr(df_new['euclidean'][50:100], y[50:100])#euclidean
# print('Correlation_eu:', round(correlation, 4))



100
100
100
100
0.07079985737800598
常態
0.34 *


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import pickle
from scipy.stats import spearmanr
from scipy import spatial
from scipy.stats import spearmanr
import numpy as np
import tensorflow.compat.v1 as tf

model = SentenceTransformer('/content/drive/MyDrive/model/SBERT_word_e1')
#model = SentenceTransformer('bert-base-uncased')
#model = SentenceTransformer('dmis-lab/biosyn-sapbert-bc5cdr-disease')
#model = SentenceTransformer('dmis-lab/biobert-base-cased-v1.2')
#model = SentenceTransformer('all-mpnet-base-v2')
#model = SentenceTransformer('emilyalsentzer/Bio_ClinicalBERT')
#model = SentenceTransformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract')

# Two lists of sentences
df= pd.read_csv('/content/drive/MyDrive/ans.csv',encoding='big5')#pair_t2d_fin

sentences1 = df['pair1'].tolist()
sentences2 = df['pair2'].tolist()


#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)



#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)


cos=[]
lst_eu=[]
#Output the pairs with their score
for i in range(len(sentences1)):
    #print("{} \t\t {} \t\t COS Score: {:.4f}".format(sentences2[i], sentences1[i], cosine_scores[i][i]))
    cos.append(float(cosine_scores[i][i]))

print(len(cos))




y = df['mean'].tolist()
correlation, p = spearmanr(cos, y)#cosine
print('Correlation_cos:', round(correlation, 2))





100
Correlation_cos: 0.61


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import pickle
from scipy.stats import spearmanr
model = SentenceTransformer('all-mpnet-base-v2')
# Two lists of sentences
df= pd.read_csv('/content/drive/MyDrive/pair_t2d_fin.csv',encoding='big5')#pair_t2d_fin

sentences1 = df['word_d_pubmed'].tolist()
sentences2 = df['word_t'].tolist()


#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cos=[]
#Output the pairs with their score
for i in range(len(sentences1)):
    #print("{} \t\t {} \t\t COS Score: {:.4f}".format(sentences2[i], sentences1[i], cosine_scores[i][i]))
    cos.append(float(cosine_scores[i][i]))
print(len(cos))

df1= pd.read_csv('/content/drive/MyDrive/td_ans.csv')
y = df1['mean'].tolist()

correlation, p = spearmanr(cos, y)#cosine
print('Correlation:', correlation)


50
Correlation: 0.46506167001131626


In [ ]:
dict = {'doc_t':sentences2, 'doc_s':sentences1,'cosine':cos} 
df_pair = pd.DataFrame(dict) 
pickle.dump( df_pair, open( "sbert_pubmed_word_t2d.p", "wb" ) )

In [ ]:
cos=[0.2974, 0.6394, 0.7026, 0.2403, 0.9838]
for i in range(len(sentences1)):
    tmp=float(cosine_scores[i][i])
    print(round(tmp-cos[i],4))
    

0.6462
0.2067
0.1701
0.3211
-0.0772


In [ ]:
from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('emilyalsentzer/Bio_ClinicalBERT')
model = SentenceTransformer('all-mpnet-base-v2')
# Two lists of sentences
df= pd.read_csv('pair_t2d_fin.csv',encoding='big5')#pair_t2d_fin

sentences1 = df['word_d_pubmed'].tolist()
sentences2 = df['word_t'].tolist()


#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cos=[]
#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t COS Score: {:.4f}".format(sentences2[i], sentences1[i], cosine_scores[i][i]))
    cos.append(float(cosine_scores[i][i]))
print(len(cos))

Electrocardiogram 		 Heart failure 		 COS Score: 0.3812
Sodium 		 Chronic kidney disease 		 COS Score: 0.2690
Computed Tomography 		 Ascites 		 COS Score: 0.1964
Blood Potassium 		 cardiac arrhythmias 		 COS Score: 0.3469
Colonoscopy 		 Colon cancer 		 COS Score: 0.7004
Electroencephalogram 		 Epilepsy 		 COS Score: 0.5900
Magnetic Resonance Imaging 		 Lung cancer 		 COS Score: 0.2764
Upper endoscopy 		 Esophageal cancer 		 COS Score: 0.4096
TSH 		 Unspecified dementia 		 COS Score: 0.1188
Carbohydrate Antigen 		 Colon cancer 		 COS Score: 0.1552
Ammonia 		 Cirrhosis 		 COS Score: 0.3268
albumin 		 Cirrhosis 		 COS Score: 0.3091
Liver Function Tests 		 Viral hepatitis 		 COS Score: 0.3370
White Blood Cell Count 		 Epilepsy 		 COS Score: 0.0769
Upper endoscopy 		 Iron deficiency anaemia 		 COS Score: 0.1863
Blood gas 		 Chronic kidney disease 		 COS Score: 0.2440
ultrasound 		 Urinary retention 		 COS Score: 0.1626
White Blood Cell Count 		 Decubitus ulcer 		 COS Score: 0.0835
Upper end

In [ ]:
from scipy.stats import spearmanr

df1= pd.read_csv('temp_reD.csv')
y = df1['mean'].tolist()

correlation, p = spearmanr(cos, y)#cosine
print('Correlation:', correlation)

Correlation: 0.4963525954162061


In [ ]:
from sentence_transformers import models, SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
model = SentenceTransformer('emilyalsentzer/Bio_ClinicalBERT')#distilbert-base-nli-mean-tokens

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=50, output_path='./sbert_test')# epochs建議2-4 太大會過擬合 

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassif

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/18375 [00:00<?, ?it/s]

In [ ]:
from scipy.stats import spearmanr

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract')
# Two lists of sentences
df= pd.read_csv('pair_t2d_fin.csv',encoding='big5')#pair_t2d_fin

sentences1 = df['word_d_pubmed'].tolist()
sentences2 = df['word_t'].tolist()


#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cos=[]
#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t COS Score: {:.4f}".format(sentences2[i], sentences1[i], cosine_scores[i][i]))
    cos.append(float(cosine_scores[i][i]))
print(len(cos))


df1= pd.read_csv('temp_reD.csv')
y = df1['mean'].tolist()

correlation, p = spearmanr(cos, y)#cosine
print('Correlation:', correlation)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/225k [00:00<?, ?B/s]

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that

Electrocardiogram 		 Heart failure 		 COS Score: 0.9358
Sodium 		 Chronic kidney disease 		 COS Score: 0.9004
Computed Tomography 		 Ascites 		 COS Score: 0.9022
Blood Potassium 		 cardiac arrhythmias 		 COS Score: 0.9016
Colonoscopy 		 Colon cancer 		 COS Score: 0.9356
Electroencephalogram 		 Epilepsy 		 COS Score: 0.9852
Magnetic Resonance Imaging 		 Lung cancer 		 COS Score: 0.8947
Upper endoscopy 		 Esophageal cancer 		 COS Score: 0.9185
TSH 		 Unspecified dementia 		 COS Score: 0.8903
Carbohydrate Antigen 		 Colon cancer 		 COS Score: 0.9186
Ammonia 		 Cirrhosis 		 COS Score: 0.9762
albumin 		 Cirrhosis 		 COS Score: 0.9739
Liver Function Tests 		 Viral hepatitis 		 COS Score: 0.9301
White Blood Cell Count 		 Epilepsy 		 COS Score: 0.8422
Upper endoscopy 		 Iron deficiency anaemia 		 COS Score: 0.9205
Blood gas 		 Chronic kidney disease 		 COS Score: 0.9050
ultrasound 		 Urinary retention 		 COS Score: 0.9130
White Blood Cell Count 		 Decubitus ulcer 		 COS Score: 0.8305
Upper end

In [ ]:
from scipy.stats import spearmanr

from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('emilyalsentzer/Bio_ClinicalBERT')
model = SentenceTransformer('emilyalsentzer/Bio_ClinicalBERT')
# Two lists of sentences
df= pd.read_csv('pair_t2d_fin.csv',encoding='big5')#pair_t2d_fin

sentences1 = df['word_d_pubmed'].tolist()
sentences2 = df['word_t'].tolist()


#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cos=[]
#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t COS Score: {:.4f}".format(sentences2[i], sentences1[i], cosine_scores[i][i]))
    cos.append(float(cosine_scores[i][i]))
print(len(cos))


df1= pd.read_csv('temp_reD.csv')
y = df1['mean'].tolist()

correlation, p = spearmanr(cos, y)#cosine
print('Correlation:', correlation)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassif

Electrocardiogram 		 Heart failure 		 COS Score: 0.7960
Sodium 		 Chronic kidney disease 		 COS Score: 0.6717
Computed Tomography 		 Ascites 		 COS Score: 0.7811
Blood Potassium 		 cardiac arrhythmias 		 COS Score: 0.6860
Colonoscopy 		 Colon cancer 		 COS Score: 0.9029
Electroencephalogram 		 Epilepsy 		 COS Score: 0.8576
Magnetic Resonance Imaging 		 Lung cancer 		 COS Score: 0.7660
Upper endoscopy 		 Esophageal cancer 		 COS Score: 0.8739
TSH 		 Unspecified dementia 		 COS Score: 0.7204
Carbohydrate Antigen 		 Colon cancer 		 COS Score: 0.7892
Ammonia 		 Cirrhosis 		 COS Score: 0.7025
albumin 		 Cirrhosis 		 COS Score: 0.7234
Liver Function Tests 		 Viral hepatitis 		 COS Score: 0.8587
White Blood Cell Count 		 Epilepsy 		 COS Score: 0.7391
Upper endoscopy 		 Iron deficiency anaemia 		 COS Score: 0.8103
Blood gas 		 Chronic kidney disease 		 COS Score: 0.7469
ultrasound 		 Urinary retention 		 COS Score: 0.7590
White Blood Cell Count 		 Decubitus ulcer 		 COS Score: 0.7595
Upper end